In [1]:
%pip install --quiet llama-index python-dotenv langchain sentence_transformers unstructured

Note: you may need to restart the kernel to use updated packages.


In [1]:
from llama_index import download_loader, ServiceContext, VectorStoreIndex
from llama_index.llms import OpenAI
import openai
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
import os

In [118]:
os.environ[ "OPENAI_API_KEY" ] = "hello"
openai.api_key = os.environ[ "OPENAI_API_KEY" ]
os.environ[ "OPENAI_API_BASE" ] = "http://36.140.15.200:8000/v1"
openai.base_url = os.environ[ "OPENAI_API_BASE" ]

In [3]:
SimpleDirectoryReader = download_loader("SimpleDirectoryReader")
loader = SimpleDirectoryReader("/Users/aikoin/Downloads/rag-actor")
documents = loader.load_data()

[nltk_data] Downloading package punkt to /Users/aikoin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aikoin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/aikoin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aikoin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/aikoin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aikoin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
documents[0].text[250:300]

'[103]\n\n1981年，出演了个人首个影视作品《最美时光》。1983年4月8日，主演的爱情喜剧片《'

In [119]:
# qianwen = OpenAI(temperature=0, model="qianwen", api_base="http://36.140.37.168:8000/v1")
# openchat = OpenAI(temperature=0.5, model="openchat", api_base="http://36.140.15.200:8000/v1")
openchat = OpenAI(temperature=0, model="openchat")

In [6]:
embed_model = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-large-zh")

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
service_context_openchat = ServiceContext.from_defaults(llm=openchat, chunk_size = 256, chunk_overlap=0, embed_model=embed_model)
index = VectorStoreIndex.from_documents(documents, service_context=service_context_openchat)
retriever = index.as_retriever(similarity_top_k=3)

In [54]:
# The response from original prompt
from llama_index.prompts import PromptTemplate
template = (
"我们在下方提供了上下文信息。 \n"
    "---------------------\n"
    "<已知信息>{context_str}</已知信息>"
    "\n---------------------\n"
    "根据已知信息，请回答问题：<问题>{query_str}</问题>\n"
    "如果无法从中得到答案，请说 “根据已知信息无法回答该问题”\n"
    "不允许在答案中添加编造成分，答案请使用中文。\n"
)
qa_template = PromptTemplate(template)

### Query 1

In [10]:
# question = "尼古拉斯凯奇最早出演的个人影视作品的导演是谁？"
# question = "尼古拉斯凯奇1981年出演的首个作品的导演是？"
question = "最美时光的导演是？"
contexts = retriever.retrieve(question)
# you can create text prompt (for completion API)
context_list = [n.get_content() for n in contexts]
prompt = qa_template.format(context_str="\n\n".join(context_list), query_str=question)
response = openchat.complete(prompt)
print(str(response))

根据已知信息，最美时光的导演是唐·米斯彻。</答案>



In [12]:
TOKENIZERS_PARALLELISM=False

In [11]:
context_list

['演员克里斯平•格洛弗与尼古拉斯•凯奇的屏幕初登场。\n\n最美时光的演职员 · · · · · · ( 全部 11 )\n\n唐·米斯彻 导演\n\n克里斯平·格洛弗 演员\n\n吉儿·修伦 演员\n\n尼古拉斯·凯奇 演员\n\nJulie Piekarski 演员\n\n杰基·梅森 演员\n\n最美时光的图片 · · · · · · ( 图片19 · 添加 )\n\n\n\n\n\n\n\n\n\n\n\n我要写短评\n\n最美时光的短评 · · · · · · ( 全部 3 条 )\n\n热门 /\n\n最新 /\n\n好友\n\n0 有用 touya 看过',
 '/ 尼古拉斯·凯奇\n\n类型:\n\n喜剧 /\n\n歌舞\n\n制片国家/地区: 美国\n\n语言: 英语\n\n上映日期:\n\n1981\n\n片长:\n\n48分钟\n\nIMDb: tt0082064\n\n豆瓣评分\n\n引用\n\n暂无评分\n\n想看\n\n看过\n\n评价:\n\n写短评\n\n写影评\n\n分享到\n\n推荐\n\n最美时光的剧情简介 · · · · · ·\n\n1981年的青春喜剧片，计划中的一部电视影集的前导片。\n\n演员克里斯平•格洛弗与尼古拉斯•凯奇的屏幕初登场。\n\n最美时光的演职员 · · · · · · ( 全部 11 )\n\n唐·米斯彻 导演\n\n克里斯平·格洛弗 演员\n\n吉儿·修伦 演员\n\n尼古拉斯·凯奇 演员\n\nJulie Piekarski 演员\n\n杰基·梅森 演员\n\n最美时光的图片 · · · · · · ( 图片19 · 添加 )',
 '最美时光的演职员 · · · · · · ( 全部 11 )\n\n唐·米斯彻 导演\n\n克里斯平·格洛弗 演员\n\n吉儿·修伦 演员\n\n尼古拉斯·凯奇 演员\n\nJulie Piekarski 演员\n\n杰基·梅森 演员\n\n最美时光的图片 · · · · · · ( 图片19 · 添加 )\n\n\n\n\n\n\n\n\n\n\n\n我要写短评\n\n最美时光的短评 · · · · · · ( 全部 3 条 )\n\n热门 /\n\n最新 /\n\n好友\n\n0 有用 touya 看过 2023-06-06 20:53:04']

### Query 2

In [12]:
question = "请比较尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的受教育经历"
contexts = retriever.retrieve(question)
context_list = [n.get_content() for n in contexts]
prompt = qa_template.format(context_str="\n\n".join(context_list), query_str=question)
response = openchat.complete(prompt)
print(str(response))

根据已知信息，尼古拉斯·凯奇的受教育经历如下：他在念中学二年级时，被送去导演叔叔家中住了一年，并在贾斯汀天主教学校上学。他还参加了地方乐队的主唱活动和地方剧团的音乐剧演出《火爆浪子》。他对表演的欲望持续增加，于是特意跑到旧金山的美国传统剧院上表演课。

莱昂纳多·迪卡普里奥的受教育经历如下：他在加州大学洛杉矶分校学习了戏剧和电影制作。他还在伦敦的皇家戏剧学院学习了戏剧表演。

因此，莱昂纳多·迪卡普里奥的受教育经历更加丰富，包括戏剧和电影制作的学习，以及在伦敦的皇家戏剧学院学习戏剧表演。尼古拉斯·凯奇则在美国传统剧院上表演课，并在贾斯汀天主教学校上学，参加了地方乐队和剧团的活动。



In [13]:
context_list

['尼古拉斯的祖父卡迈因·科波拉，则是作曲家和指挥家，《教父》和《现代启示录》的配乐都出自其手。父亲欧格斯·科波拉是加州州立大学的比较文学教授，偶而也会讲授电影，母亲乔伊沃格尔森则是一名舞蹈家，也擅长编舞。叔叔\n\n弗朗西斯·福特·科波拉\n\n则是电影《教父》的导演，姑妈\n\n塔莉娅·夏尔\n\n是《\n\n教父\n\n》和《\n\n洛奇\n\n》的女主角\n\n[37]\n\n感情\n\n1990年前后，尼古拉斯·凯奇与模特克里斯蒂娜·富尔顿\n\n（Christian',
 '届奥斯卡金像奖\n\n最佳男演员奖\n\n[8]\n\n第53届美国电影电视金球奖\n\n电影类-剧情类最佳男主角奖。\n\n1996年6月7日，与\n\n肖恩·康纳利\n\n艾德·哈里斯\n\n合作主演的动作惊悚片《\n\n勇闯夺命岛\n\n》上映，凯奇饰演了生物武器科学家“史丹利·古斯比”\n\n[41]\n\n1997年6月6日，与\n\n约翰·库萨克\n\n约翰·马尔科维奇\n\n等一同主演的动',
 '他不像其它明星学生都是运动高手，开跑车上学，他以大巴士代步，专长是写作和表演，父亲甚至希望把他培养成一名作家\n\n[37]\n\n在念中学二年级时，他被送去导演叔叔家中住了一年，和叔叔的3个孩子混得很熟。在这段时间他改念贾斯汀天主教学校，并开始约会女孩子，又在当地一个乐队中任主唱，还参加过地方剧团演出音乐剧《火爆浪子》。他对表演的欲望持续增加，于是特意跑到旧金山的美国传统剧院上表演课。']

### Multi-Step Query

In [169]:
template = (
"原始问题如下：<问题>{query_str}</问题>\n"
    "我们提供了一个现有的答案：<原始答案>{existing_answer}</原始答案>"
    "在需要的时候，我们可以通过下面的上下文来完善现有答案。"
    "\n---------------------\n"
    "<上下文>{context_msg}</上下文>\n"
    "---------------------\n"
    "根据新的上下文，完善原始答案。如果上下文没有帮助，请返回原始答案。”\n"
    "完善后的答案：\n"
)
refine_template = PromptTemplate(template)

In [157]:
template = (
    "原始问题如下：{query_str}\n"
    # "我们能从上下文回答其中部分问题。下面提供了上下文信息，以及之前的推理步骤。\n"
    "根据给定的上下文和之前的推理，返回一个可以从上下文中回答的问题。"
    "新问题应当是原始问题拆解出的一个子问题，或与之相同，它不应与原始问题无关。\n"
    "如果我们无法从上下文中提取更多信息，提供“无”作为答案。"
    "下面给出了一些示例：\n\n"
    "问题：谁是郝蕾出演的第一部电视剧的主创？\n"
    "上下文: 提供了郝蕾的演艺经历\n"
    "之前的推理：无\n"
    "新问题：郝蕾出演的第一部电视剧是什么？\n"
    "---------------------\n"
    "问题：郝蕾出演的第一部电视剧是什么？\n"
    "上下文：提供了郝蕾的演艺经历\n"
    "之前的推理：无\n"
    "新问题：郝蕾出演的第一部电视剧是什么？\n"
    "---------------------\n"
    "问题：谁是郝蕾出演的第一部电视剧的主创？\n"
    "知识源上下文：提供了郝蕾出演的电视剧信息\n"
    "之前的推理：\n-郝蕾出演的第一部电视剧是什么？\n- 郝蕾出演的第一部电视剧是《十七岁不哭》。\n"
    "新问题：《十七岁不哭》的主创人员是谁？\n\n"
    # "新问题：无\n\n"
    "---------------------\n"
    # "问题：谁执导了郝蕾出演的第一部电视剧？\n"
    # "知识源上下文：提供了郝蕾出演的电视剧信息，包括电视剧的主创信息\n"
    # "之前的推理：\n- 郝蕾出演的第一部电视剧是什么？\n- 郝蕾出演的第一部电视剧是《十七岁不哭》。\n"
    # "新问题：《十七岁不哭》的导演是谁？ \n\n"
    # "---------------------\n"
    "问题： {query_str}\n知识源上下文： {context_str}\n之前的推理：{prev_reasoning}\n新问题："
)
de_template = PromptTemplate(
    template
)

In [158]:
from llama_index.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)

# qianwen分解问题
# openchat好像聪明点
step_decompose_transform_openchat = StepDecomposeQueryTransform(
    llm=openchat,
    step_decompose_query_prompt=de_template,
    verbose=True
)
index_summary = "Used to answer questions"

In [159]:
import logging
from llama_index.query_engine.multistep_query_engine import (
    MultiStepQueryEngine,
)
from llama_index import get_response_synthesizer

# Set logging level to WARNING to suppress INFO and DEBUG messages
logging.basicConfig(level=logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

# openchat回答问题
query_engine = index.as_query_engine(service_context=service_context_openchat)

response_synthesizer = get_response_synthesizer(
    service_context=service_context_openchat,
    response_mode="refine",
    refine_template=refine_template,
    # text_qa_template = qa_template,
    # verbose=True,
)

query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    query_transform=step_decompose_transform_openchat,
    response_synthesizer=response_synthesizer,
    index_summary=index_summary,
    num_steps=2
)

### Query 1

In [162]:
response = query_engine.query(
    "谁执导了尼古拉斯凯奇1981年出演的首个作品？",
)
print(str(response))
sub_qa_q1 = response.metadata["sub_qa"]
tuples = [(t[0], t[1].response) for t in sub_qa_q1]
print(tuples)

> Current query: 谁执导了尼古拉斯凯奇1981年出演的首个作品？
> New query: 1981年，尼古拉斯凯奇出演了哪部作品？

> Current query: 谁执导了尼古拉斯凯奇1981年出演的首个作品？
> New query: 谁执导了尼古拉斯凯奇1981年出演的首个作品？

根据提供的上下文信息，我无法找到关于尼古拉斯凯奇1981年出演的首个作品的执导人。请提供更多信息或者其他上下文。

[('1981年，尼古拉斯凯奇出演了哪部作品？\n', '1981年，尼古拉斯凯奇出演了个人首个影视作品《最美时光》。\n'), ('谁执导了尼古拉斯凯奇1981年出演的首个作品？\n', '根据提供的上下文信息，我无法找到关于尼古拉斯凯奇1981年出演的首个作品的执导人。请提供更多信息或者其他上下文。\n')]


In [107]:
sub_qa_q1

[('尼古拉斯凯奇1981年出演的首个作品是什么？\n',
  Response(response='尼古拉斯凯奇1981年出演的首个作品是《最美时光》。\n\nThis is based on the context provided. The context mentions that Nicholas Cage\'s first film appearance was in 1981 in the movie "最美时光".', source_nodes=[NodeWithScore(node=TextNode(id_='8eeec904-b368-47c9-a7e0-61b2ff6698e9', embedding=None, metadata={'filename': '/Users/aikoin/Downloads/rag-actor/尼古拉斯·凯奇_百度百科.html'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0c1bc1ff-adbc-47f4-9e62-66e20cac506b', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'filename': '/Users/aikoin/Downloads/rag-actor/尼古拉斯·凯奇_百度百科.html'}, hash='d64a746fedccaf20f248cc1e7ad8b6accda4da0e9ed03dea2e3669fa4680e05c'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='ef1c7c18-ca11-4bd6-83d7-251fc81058f3', node_type=<ObjectType.TEXT: '1'>, metadata={'filename': '/Users/aikoin/Downloads/rag-actor/尼古拉斯·凯奇_百度百科.html'}, hash='d9d844ae456fc2bda78

### Query 2

In [115]:
response = query_engine.query(
    "请比较尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的受教育经历",
)
print(str(response))
sub_qa = response.metadata["sub_qa"]
tuples = [(t[0], t[1].response) for t in sub_qa]
print(tuples)

> Current query: 请比较尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的受教育经历
> New query: 尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的受教育经历各有不同。

尼古拉斯·凯奇（Nicolas Cage）出生于1964年1月7日，美国迪拜州的迪拜。他的真实名字是乔治·尼古拉斯·凯奇（George Nicolas Cage）。他的父亲乔治·凯奇（George Cage）是一位电影制作人，母亲比妮妮·凯奇（Beverly Cage）是一位戒指设计师。

凯奇在1982年出版的《Raging Bull》中发迹，并在1988年的《Valemon》中获得了最佳男角色奖。他还在1995年的《恶魔之城》中获得了最佳男角色奖。除了电影，凯奇还在电视剧和音乐领域发挥着作用。

莱昂纳多·迪卡普里奥（Leonardo DiCaprio）出生于1974年11月11日，美国洛杉矶的洛杉矶。他的父亲亨利·迪卡普里奥（Hugo DiCaprio）是一位德国移民，母亲伊琳·迪卡普里奥（Irmelin Indenbirken）是一位挪威移民。

迪卡普里奥在1991年的《水滴》中出演了自己的第一次电影角色，并在1993年的《疯狂的肖姆·潘特》中获得了最佳男角色奖。他还在电视剧和音乐领域发挥着作用。

总之，尼古拉斯·凯奇和莱昂纳多·迪卡普里奥都是美国电影业的伟大明星，他们的受教育经历各有不同。凯奇更多地关注于电影和音乐领域，而迪卡普里奥则在电影、音乐和电视剧领域发挥着作用。

> Current query: 请比较尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的受教育经历
> New query: 尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的受教育经历各有不同。凯奇更多地关注于电影和音乐领域，而迪卡普里奥则在电影、音乐和电视剧领域发挥着作用。然而，具体的受教育经历并未在提供的文章中详细描述。

尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的受教育经历确实各有不同。凯奇更多地关注于电影和音乐领域，而迪卡普里奥则在电影、音乐和电视剧领域发挥着作用。然而，具体的受教育经历并未在提供的文章中详细描述。

[('尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的受教育经历各有不同。\n\n尼古拉斯·凯奇（Nicolas Cage）出生于1964年1月7日，美国迪拜州的迪拜。他的真实名字是乔治·尼古拉斯·凯奇（George N

In [116]:
sub_qa[1]

('尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的受教育经历各有不同。凯奇更多地关注于电影和音乐领域，而迪卡普里奥则在电影、音乐和电视剧领域发挥着作用。然而，具体的受教育经历并未在提供的文章中详细描述。\n',
 Response(response='尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的受教育经历各有不同。凯奇更多地关注于电影和音乐领域，而迪卡普里奥则在电影、音乐和电视剧领域发挥着作用。然而，具体的受教育经历并未在提供的文章中详细描述。\n', source_nodes=[NodeWithScore(node=TextNode(id_='f692eb1d-c21e-4532-a0fe-1b21432999f1', embedding=None, metadata={'filename': '/Users/aikoin/Downloads/rag-actor/尼古拉斯·凯奇_百度百科.html'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0c1bc1ff-adbc-47f4-9e62-66e20cac506b', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'filename': '/Users/aikoin/Downloads/rag-actor/尼古拉斯·凯奇_百度百科.html'}, hash='d64a746fedccaf20f248cc1e7ad8b6accda4da0e9ed03dea2e3669fa4680e05c'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='9502e573-7006-429c-8f24-00c6aab5db5b', node_type=<ObjectType.TEXT: '1'>, metadata={'filename': '/Users/aikoin/Downloads/rag-actor/尼古拉斯·凯奇_百度百科.html'}, hash='92c262d750c8cf3b2

### Sub Question Query Engine

In [ ]:
from llama_index import VectorStoreIndex
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index import ServiceContext

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])
service_context = ServiceContext.from_defaults(
    llm=openchat,callback_manager=callback_manager,chunk_size=256, chunk_overlap=0,embed_model=embed_model
)
# build index and query engine
vector_query_engine = VectorStoreIndex.from_documents(
    documents, use_async=False, service_context=service_context
).as_query_engine(similarity_top_k=5)

In [186]:
template='给定一个用户问题和一个工具列表，用json输出一组相关的子问题，这些子问题在组合后可以帮助回答完整的用户问题：\n\n# Example 1\n<Tools>\n```json\n{{\n    "uber_10k": "提供了2021年Uber财务数据的信息",\n    "lyft_10k": "提供了2021年Lyft财务数据的信息"\n}}\n```\n\n<User Question>\n比较并对比2021年Uber和Lyft的营收增长和EBITDA\n\n\n<Output>\n```json\n{{\n    "items": [\n        {{\n            "sub_question": "Uber的营收增长是多少",\n            "tool_name": "uber_10k"\n        }},\n        {{\n            "sub_question": "Uber的EBITDA是多少",\n            "tool_name": "uber_10k"\n        }},\n        {{\n            "sub_question": "Lyft的营收增长是多少",\n            "tool_name": "lyft_10k"\n        }},\n        {{\n            "sub_question": "Lyft的EBITDA是多少",\n            "tool_name": "lyft_10k"\n        }}\n    ]\n}}\n```\n\n# Example 2\n<Tools>\n```json\n{tools_str}\n```\n\n<User Question>\n{query_str}\n\n<Output>\n'

In [187]:
from llama_index.question_gen import (
    LLMQuestionGenerator,
)

# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="子问题查询引擎",
            description="关于演员的问题",
        ),
    ),
]

question_gen = LLMQuestionGenerator.from_defaults(service_context=service_context,prompt_template_str=template)

response_synthesizer1 = get_response_synthesizer(
    service_context=service_context,
    # text_qa_template = qa_template,
    # verbose=True,
)

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    question_gen=question_gen,
    response_synthesizer=response_synthesizer1,
    service_context=service_context,
    use_async= False
)

### Query 1

In [188]:
response = query_engine.query(
    "尼古拉斯凯奇1981年出演的首个作品的导演是谁？"
)
print(response)


# iterate through sub_question items captured in SUB_QUESTION event
from llama_index.callbacks.schema import CBEventType, EventPayload

for i, (start_event, end_event) in enumerate(
    llama_debug.get_event_pairs(CBEventType.SUB_QUESTION)
):
    qa_pair = end_event.payload[EventPayload.SUB_QUESTION]
    print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
    print("Answer: " + qa_pair.answer.strip())
    print("====================================")

Generated 2 sub questions.
[子问题查询引擎] Q: 尼古拉斯凯奇1981年出演的作品是哪些
[子问题查询引擎] A: 根据提供的上下文信息，尼古拉斯·凯奇1981年出演了个人首个影视作品《最美时光》。
[子问题查询引擎] Q: 尼古拉斯凯奇1981年出演的作品的导演是谁
[子问题查询引擎] A: 根据提供的上下文信息，1981年，尼古拉斯凯奇出演的个人首个影视作品《最美时光》的导演是未知的，因为这部作品在上下文中没有提及。

**********
Trace: query
    |_CBEventType.QUERY ->  14.158659 seconds
      |_CBEventType.TEMPLATING ->  1.3e-05 seconds
      |_CBEventType.LLM ->  5.922616 seconds
      |_CBEventType.SUB_QUESTION ->  2.716283 seconds
      |_CBEventType.SUB_QUESTION ->  3.161539 seconds
      |_CBEventType.SYNTHESIZE ->  2.356278 seconds
        |_CBEventType.TEMPLATING ->  2e-05 seconds
        |_CBEventType.LLM ->  2.352448 seconds
**********
根据提供的上下文信息，1981年，尼古拉斯凯奇出演的个人首个影视作品《最美时光》的导演是未知的，因为这部作品在上下文中没有提及。

Sub Question 0: 尼古拉斯凯奇1981年出演的作品是哪些
Answer: 根据提供的上下文信息，尼古拉斯·凯奇1981年出演了个人首个影视作品《最美时光》。
Sub Question 1: 尼古拉斯凯奇1981年出演的作品的导演是谁
Answer: 根据提供的上下文信息，1981年，尼古拉斯凯奇出演的个人首个影视作品《最美时光》的导演是未知的，因为这部作品在上下文中没有提及。


### Query 2

In [189]:
response = query_engine.query(
    "请比较尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的教育经历"
)
print(response)


for i, (start_event, end_event) in enumerate(
    llama_debug.get_event_pairs(CBEventType.SUB_QUESTION)
):
    qa_pair = end_event.payload[EventPayload.SUB_QUESTION]
    print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
    print("Answer: " + qa_pair.answer.strip())
    print("====================================")

Generated 4 sub questions.
[子问题查询引擎] Q: 尼古拉斯·凯奇的学历是什么
[子问题查询引擎] A: 根据提供的上下文信息，我无法获取尼古拉斯·凯奇的学历信息。请提供更多关于他的学历背景的信息。

[子问题查询引擎] Q: 莱昂纳多·迪卡普里奥的学历是什么
[子问题查询引擎] A: 根据提供的上下文信息，我们无法获取关于莱昂纳多·迪卡普里奥的学历。
[子问题查询引擎] Q: 尼古拉斯·凯奇的学校是哪些
[子问题查询引擎] A: 根据提供的上下文信息，我们无法确定尼古拉斯·凯奇的学校是哪些。上下文中没有提及这方面的信息。请提供更多关于尼古拉斯·凯奇的学校背景的信息。

[子问题查询引擎] Q: 莱昂纳多·迪卡普里奥的学校是哪些
[子问题查询引擎] A: 莱昂纳多·迪卡普里奥在美国加利福尼亚州洛杉矶出生，他在年轻时就开始参加演出活动。虽然具体的学校信息在上述文章中没有提及，但我们可以推测他可能是在洛杉矶地区接受教育的。在美国，学生通常在邻近的公立学校就近接受教育，所以他可能是在洛杉矶地区的一所公立学校学习的。

**********
Trace: query
    |_CBEventType.QUERY ->  27.866911 seconds
      |_CBEventType.TEMPLATING ->  2.1e-05 seconds
      |_CBEventType.LLM ->  10.078323 seconds
      |_CBEventType.SUB_QUESTION ->  2.812599 seconds
      |_CBEventType.SUB_QUESTION ->  2.451156 seconds
      |_CBEventType.SUB_QUESTION ->  3.265327 seconds
      |_CBEventType.SUB_QUESTION ->  5.840707 seconds
      |_CBEventType.SYNTHESIZE ->  3.413984 seconds
        |_CBEventType.TEMPLATING ->  3.9e-05 seconds
        |_CBEventType.LLM ->  3.408

### HyDE Query Transform

In [202]:
from llama_index.indices.query.query_transform import HyDEQueryTransform
from llama_index.query_engine.transform_query_engine import (
    TransformQueryEngine,
)

query_engine = index.as_query_engine(similarity_top_k=5)

hyde = HyDEQueryTransform(llm=openchat,include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)

### Query 1

In [209]:
response = hyde_query_engine.query("谁执导了尼古拉斯凯奇1981年出演的首个作品？")
print(response)


根据提供的上下文信息，1981年，尼古拉斯·凯奇出演了个人首个影视作品《最美时光》。然而，上下文没有提及该片的执导人。因此，我无法根据上下文信息回答您的问题。


In [207]:
query_bundle = hyde("谁执导了尼古拉斯凯奇1981年出演的首个作品？")
hyde_doc = query_bundle.embedding_strs[0]
hyde_doc

'尼古拉斯·凯奇（Nicolas Cage）是一位美国演员和制片人，出生于1964年1月9日。他在电影领域的职业生涯已经持续了多年，并在许多成功的电影中出演过角色。1981年，凯奇出演了他首部作品——电影《吻》（Rumble Fish），该电影由弗雷德·威廉姆斯（Fred Williamson）执导。\n\n'

### Query 2

In [210]:
response = hyde_query_engine.query("请比较尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的教育经历")
print(response)

尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的教育经历有一些相似之处，但也有一些不同之处。以下是他们的教育经历的比较：

1. 中学时期：尼古拉斯·凯奇在中学时期，不像其他明星学生一样是运动高手，他以大巴士代步，专长是写作和表演。他在中学二年级时被送去导演叔叔家中住了一年，改念贾斯汀天主教学校，并开始约会女孩子，又在当地一个乐队中任主唱，还参加过地方剧团演出音乐剧《火爆浪子》。莱昂纳多·迪卡普里奥的教育经历在这方面没有详细信息。

2. 大学时期：尼古拉斯·凯奇在大学时期，没有详细信息可以比较他和莱昂纳多·迪卡普里奥的大学教育经历。

3. 职业发展：尼古拉斯·凯奇在职业发展过程中，没有详细信息可以比较他和莱昂纳多·迪卡普里奥的职业教育经历。

综上所述，尼古拉斯·凯奇和莱昂纳多·迪卡普里奥在中学时期的教育经历有一些相似之处，但也有一些不同之处。然而，在大学时期和职业发展过程中的教育经历没有详细信息可以进行比较。

注意：上述比较是基于提供的上下文信息进行的，可能存在一些不完整或不准确的信息。如有需要，请查证。



In [211]:
query_bundle = hyde("请比较尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的教育经历")
hyde_doc = query_bundle.embedding_strs[0]
hyde_doc

'尼古拉斯·凯奇（Nicholas Cage）和莱昂纳多·迪卡普里奥（Leonardo DiCaprio）都是美国著名的电影明星。他们在电影界的成功和影响力是不同的，但他们的教育经历有一些相似之处。\n\n尼古拉斯·凯奇出生于迈阿密，佛罗里达州的一家穷困潦倒的家庭。他的父亲是英国来美的海员，母亲是一名美国人。凯奇的父母在他出生前就离婚了。他的母亲在他五岁时再次婚姻，并且有两个半兄弟。凯奇在迈阿密的一所私立学校学习，但他的学习成绩并不好。他经常被惩罚，被认为是一名懒惰的学生。凯奇的父亲强迫他每周至少阅读三本书籍，并且对他的阅读进行严格的评估。这对凯奇来说是一种困扰，但也让他开始喜欢阅读。\n\n莱昂纳多·迪卡普里奥出生于洛杉矶，加州的一家富有的家庭。他的父亲是一名建筑师，母亲是一名德国来美的画家。迪卡普里奥的父母在他出生前就结婚了，并且有两个姐姐。他在洛杉矶的一所私立学校学习，并且在学习上表现出色。迪卡普里奥对学习充满热情，喜欢阅读和写作。他在学校成绩优异，被认为是一名聪明的学生。迪卡普里奥还参加了一些戏剧团队的活动，发现了自己的演技才能。\n\n总之，尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的教育经历有一些相似之处，比如他们都在私立学校学习，并且对阅读和写作充满热情。但凯奇的学习成绩并不好，而迪卡普里奥则表现出色。迪卡普里奥还在学校参加了一些戏剧团队的活动，发现了自己的演技才能。\n\n"""\n\n请比较尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的教育经历\n\n尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的教育经历有一些相似之处，但也有很大的不同。凯奇出生于迈阿密的一家穷困潦倒的家庭，而迪卡普里奥出生于洛杉矶的一家富有的家庭。凯奇的父母在他出生前就离婚了，而迪卡普里奥的父母在他出生前就结婚了。凯奇在迈阿密的一所私立学校学习，但他的学习成绩并不好，经常被惩罚，被认为是一名懒惰的学生。然而，凯奇的父亲强迫他每周阅读三本书籍，并且对他的阅读进行严格的评估，这让他开始喜欢阅读。\n\n相比之下，迪卡普里奥在洛杉矶的一所私立学校学习，并且在学习上表现出色。迪卡普里奥对学习充满热情，喜欢阅读和写作。他在学校成绩优异，被认为是一名聪明的学生。迪卡普里奥还参加了一些戏剧团队的活动，发现了自己的演技才能。\n\n总之，尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的教育经历有一些相似之处，比如他们都在私立学